In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
from pathlib import Path

import sisl
from sisl import viz




# Global parameters

In [2]:
BOND = 1.42
VACUUM = 3
KIND = "zigzag"

# Create directories for the files.

In [3]:

structure_directory = "structures"
current_working_directory = Path()
structure_path = current_working_directory / structure_directory
structure_path.mkdir(exist_ok=True, parents=True)

# Create structure

In [4]:
graphene = sisl.geom.graphene_nanoribbon(width=3, kind="zigzag", vacuum=1)
graphene.write(structure_path / "graphene.xyz") 
# graphene.plot()

## Reading data

In [5]:
test1 = sisl.Geometry.new(structure_path / "graphene.xyz") # read data or python object
test2 = sisl.io.get_sile(structure_path / "graphene.xyz") # 
test3 = sisl.Geometry.read(structure_path / "graphene.xyz") # calls the same as above
print(test1.lattice)
print()
print(test2.read_lattice())
print()
print(test3.lattice)

Lattice{nsc: [3 1 1],
 origin=[0.0000, 0.0000, 0.0000],
 A=[2.4595, 0.0000, 0.0000],
 B=[0.0000, 5.9700, 0.0000],
 C=[0.0000, 0.0000, 1.0000],
 bc=[Periodic,
     Unknown,
     Unknown]
}

Lattice{nsc: [3 1 1],
 origin=[0.0000, 0.0000, 0.0000],
 A=[2.4595, 0.0000, 0.0000],
 B=[0.0000, 5.9700, 0.0000],
 C=[0.0000, 0.0000, 1.0000],
 bc=[Periodic,
     Unknown,
     Unknown]
}

Lattice{nsc: [3 1 1],
 origin=[0.0000, 0.0000, 0.0000],
 A=[2.4595, 0.0000, 0.0000],
 B=[0.0000, 5.9700, 0.0000],
 C=[0.0000, 0.0000, 1.0000],
 bc=[Periodic,
     Unknown,
     Unknown]
}


## Adding atoms to structure

In [6]:
graphene = sisl.geom.graphene_nanoribbon(width=3, kind="zigzag", vacuum=1)
print(graphene.atoms)
graphene += graphene
print(graphene.atoms)


Atoms{species: 1,
 Atom{C, Z: 6, mass(au): 12.01070, maxR: 1.43420,
  Orbital{R: 1.43420, q0: 0.0}
 }: 6,
}
Atoms{species: 1,
 Atom{C, Z: 6, mass(au): 12.01070, maxR: 1.43420,
  Orbital{R: 1.43420, q0: 0.0}
 }: 12,
}


In [7]:
width = 3 # num of C-C bonds
graphene_arm = sisl.geom.graphene_nanoribbon(width=width, bond=BOND, kind=KIND, vacuum=VACUUM)
graphene_arm.write(structure_path / "graphene_arm.xyz")
graphene_arm.plot()

## Solution generated in part by chatGPT

Workflow:
- Generate hexagonal carbon strucure `ring`
- Generate carbon nanoribbon `arm` of some length $N$ (will be $ N = 2M$ with $M$ number of C-C bonds)
- Determine center of structure `ring` and vector from center to `ring`-atom $i$ called `v`
- Angle between each atom is $\theta = 60^\circ$
- Rotate arm by $\theta$ w.r.t. previous atom
- Translate `arm` by $a\frac{\vec{v}}{||\vec{v}||}$ from `ring`-atom $i$ and first atom of `arm` using bond length $a = 1.42$
- Add arm to `ring` atom $i$

In [22]:
# 1. Hexagonal carbon ring
ring = sisl.geom.graphene_flake(shells=0, bond=BOND, atoms="C", vacuum=VACUUM)

# 2. Arm to attach (here zigzag nanoribbon of length n)
arm = sisl.geom.graphene_nanoribbon(2, BOND, kind="armchair", vacuum=VACUUM)
# 3. Attach one arm to each of the 6 ring atoms
system = ring.copy()
center = np.mean(ring.xyz, axis=0)
for i in ring:
    # outward radial direction
    v = ring[i] - center
    # angle from +x axis
    angle = i*60
    # rotate a copy of the arm so its axis aligns with v
    rotated = arm.rotate(angle, v=[0,0,1])
    # translate so first atom of rotated arm is bond length away
    shift = ring[i] + v/np.linalg.norm(v)*BOND - rotated[0]
    rotated = rotated.translate(shift)
    # extend system
    system += rotated
# print(system.atoms)
print(f"""
arms : 6 x 4
ring : 6
total: {len(ring)*len(arm) + len(ring)}
""")

# 4. Plot
system.plot()



arms : 6 x 4
ring : 6
total: 30



In [9]:
viz.geometry_plot(system, axes=["x", "y"])

### Save new structure

In [23]:
system.write(structure_path / "trial.xyz")